In [1]:
import keras




Using TensorFlow backend.


In [2]:
# load data
import pandas as pd
import numpy as np
from scipy.ndimage import imread
from scipy.misc import imresize
SAMPLE_SUB = 'leaf_data/sample_submission.csv'
TRAIN_CSV = 'leaf_data/train.csv'
TEST_CSV = 'leaf_data/test.csv'

IMG_SIZE = (256,256)

sample_df = pd.read_csv(SAMPLE_SUB)
res_cols = sample_df.columns.values[1:]
print res_cols
col_cnt = len(res_cols)

train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
print train_df.head()
print test_df.head()

['Acer_Capillipes' 'Acer_Circinatum' 'Acer_Mono' 'Acer_Opalus'
 'Acer_Palmatum' 'Acer_Pictum' 'Acer_Platanoids' 'Acer_Rubrum'
 'Acer_Rufinerve' 'Acer_Saccharinum' 'Alnus_Cordata' 'Alnus_Maximowiczii'
 'Alnus_Rubra' 'Alnus_Sieboldiana' 'Alnus_Viridis' 'Arundinaria_Simonii'
 'Betula_Austrosinensis' 'Betula_Pendula' 'Callicarpa_Bodinieri'
 'Castanea_Sativa' 'Celtis_Koraiensis' 'Cercis_Siliquastrum'
 'Cornus_Chinensis' 'Cornus_Controversa' 'Cornus_Macrophylla'
 'Cotinus_Coggygria' 'Crataegus_Monogyna' 'Cytisus_Battandieri'
 'Eucalyptus_Glaucescens' 'Eucalyptus_Neglecta' 'Eucalyptus_Urnigera'
 'Fagus_Sylvatica' 'Ginkgo_Biloba' 'Ilex_Aquifolium' 'Ilex_Cornuta'
 'Liquidambar_Styraciflua' 'Liriodendron_Tulipifera'
 'Lithocarpus_Cleistocarpus' 'Lithocarpus_Edulis' 'Magnolia_Heptapeta'
 'Magnolia_Salicifolia' 'Morus_Nigra' 'Olea_Europaea' 'Phildelphus'
 'Populus_Adenopoda' 'Populus_Grandidentata' 'Populus_Nigra' 'Prunus_Avium'
 'Prunus_X_Shmittii' 'Pterocarya_Stenoptera' 'Quercus_Afares'
 'Querc

In [3]:
def make_train_feature(id):
    p = 'leaf_data/images/{}.jpg'.format(int(id))
    img_data = imread(p)
    img_data = imresize(img_data,IMG_SIZE)
    csv_f = train_df[train_df.id==id].values[0][2:]
    return img_data,csv_f

def make_test_feature(id):
    p = 'leaf_data/images/{}.jpg'.format(int(id))
    img_data = imread(p)
    img_data = imresize(img_data,IMG_SIZE)
    csv_f = test_df[test_df.id==id].values[0][1:]
    return img_data,csv_f
    
def trans_y(cls):
    y = np.zeros(col_cnt)
    idx = np.where(res_cols==cls)
    y[idx] += 1
    return y

print make_train_feature(1)
print trans_y('Acer_Capillipes')

(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), array([0.007812, 0.023438, 0.023438, 0.003906, 0.011719, 0.009766,
       0.027344, 0.0, 0.001953, 0.033202999999999996, 0.013672, 0.019531,
       0.06640599999999999, 0.0, 0.029297000000000004, 0.0, 0.03125,
       0.011719, 0.0, 0.025391, 0.023438, 0.001953, 0.0, 0.015625, 0.0,
       0.03125, 0.0, 0.013672, 0.029297000000000004, 0.015625, 0.011719,
       0.003906, 0.025391, 0.0, 0.001953, 0.011719, 0.009766,
       0.041016000000000004, 0.037108999999999996, 0.019531, 0.0, 0.009766,
       0.021484, 0.015625, 0.007812, 0.013672, 0.027344, 0.0625, 0.0,
       0.017578, 0.03125, 0.0, 0.044922000000000004, 0.007812, 0.025391,
       0.003906, 0.013672, 0.015625, 0.013672, 0.003906,
       0.0058590000000000005, 0.003906, 0.019531, 0.001953, 0.00064671,
       0.00060

In [7]:
# just flat
x_train = []
y_train = []
x_test = []

for id in train_df.id.values:
    img,csv_f = make_train_feature(id)
    x_train.append(np.concatenate((img.flatten(),csv_f,np.zeros(64))))

In [9]:
x_train = np.array(x_train)
print x_train.shape
x_train = np.reshape(x_train,(-1,256,257,1))
print x_train.shape

(990, 65792)
(990, 256, 257, 1)


In [10]:
y_train = [trans_y(cls) for cls in train_df.species.values]

for id in test_df.id.values:
    img,csv_f = make_test_feature(id)
    x_test.append(np.concatenate((img.flatten(),csv_f,np.zeros(64))))

In [11]:
y_train = np.array(y_train)
x_test = np.array(x_test)
x_test = np.reshape(x_test,(-1,256,257,1))
print x_train.shape
print y_train.shape
print x_test.shape

(990, 256, 257, 1)
(990, 99)
(594, 256, 257, 1)


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Conv2D,Flatten,MaxPooling2D,Dropout

def get_simple_model():
    model = Sequential()
    
    # conv1
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(256,257,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.25))
    
    # conv2
    model.add(Conv2D(filters=32,kernel_size=(5,5)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.25))
    
    # conv3
    model.add(Conv2D(filters=32,kernel_size=(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # dense1
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    
    # dense2
    model.add(Dense(99))
    model.add(Activation('softmax'))
    
    
    print 'start compile'
    model.compile(optimizer='rmsprop',
              loss='mse')
    print 'compile done'
    return model

m = get_simple_model()

start compile
compile done


In [ ]:
epo = 100
b_size = 32

m.fit(x_train,y_train,epochs=epo,batch_size=b_size,verbose=2,validation_split=0.2)
    

Train on 792 samples, validate on 198 samples
Epoch 1/100
224s - loss: 0.0200 - val_loss: 0.0199
Epoch 2/100
231s - loss: 0.0200 - val_loss: 0.0199
Epoch 3/100
217s - loss: 0.0200 - val_loss: 0.0199
Epoch 4/100
383s - loss: 0.0200 - val_loss: 0.0199
Epoch 5/100
225s - loss: 0.0200 - val_loss: 0.0199
Epoch 6/100


In [ ]:
pred = m.predict(x_test)
print pred.shape
print pred[:10]

res_df = sample_df.copy()


